# Installs

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install streamlit
!pip install wikipedia
!pip install pip install numexpr

# Imports

In [1]:
import os
# Importing the openapi_key from a file named secret_key
from secret_key import openapi_key
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain , SimpleSequentialChain , SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
import re

# Setup

### Setting the OpenAI API key as an environment variable

In [2]:
os.environ['OPENAI_API_KEY'] = openapi_key

### Initialize the language model with a temperature setting of 0.6

In [3]:
# Initializing an instance of the OpenAI class with a temperature setting of 0.6
# The temperature setting controls the randomness of the model's output
# Lower temperature means more deterministic and safer responses
# Higher temperature means more creative and varied responses
llm = OpenAI(temperature=.6)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## Single input

In [4]:
# Defining the input prompt for the model
Input = "What are the best neighborhoods to explore in Brooklyn?"

# Generating a response from the model based on the input prompt
response = llm(Input)

# Printing the generated response to the console
print(response)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




1. Williamsburg: Known for its vibrant arts and music scene, Williamsburg is a popular neighborhood for young professionals and creatives. It also has a wide variety of trendy restaurants, bars, and shops.

2. Park Slope: This family-friendly neighborhood is known for its beautiful brownstone buildings, tree-lined streets, and community feel. It also has a great selection of restaurants, cafes, and shops.

3. DUMBO: Short for Down Under the Manhattan Bridge Overpass, this neighborhood is popular for its stunning views of the Manhattan skyline and its industrial-chic vibe. It is also home to many art galleries, trendy restaurants, and the iconic Brooklyn Bridge.

4. Fort Greene: This historic neighborhood is known for its beautiful architecture, tree-lined streets, and diverse community. It also has a thriving cultural scene, with the Brooklyn Academy of Music and the Barclays Center located here.

5. Greenpoint: This waterfront neighborhood has a strong Polish influence and is known 

## Basic Prompt Template

### Define the prompt template for neighborhood suggestions

In [5]:
prompt_template_name = PromptTemplate(
    input_variables = ['city'],
    template = "What are the best neighborhoods to explore in {city}?"
)

prompt_template_name.format(city="Miami")

'What are the best neighborhoods to explore in Miami?'

### Define the LLM Chain for neighborhood suggestions

In [6]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### Prompt the user for input and run the chain

In [7]:
# Prompt the user for input
user_input = input("Enter a city: ")

# Run the chain with the user's input for the 'city' variable
response = chain.run(user_input)

# Printing the generated response to the console
print(response)

Enter a city:  Philly


C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




1. Center City: This bustling neighborhood is the heart of Philadelphia and is home to many iconic landmarks such as City Hall, Love Park, and Rittenhouse Square. It also offers a variety of shopping, dining, and entertainment options.

2. Old City: Known as the historic district, this neighborhood is where the Declaration of Independence was signed and is filled with cobblestone streets, colonial architecture, and numerous museums and galleries.

3. University City: Home to the University of Pennsylvania and Drexel University, this neighborhood has a vibrant and youthful atmosphere with plenty of trendy restaurants, cafes, and shops.

4. Fishtown: This up-and-coming neighborhood has a hip and artsy vibe with a mix of old and new establishments, including galleries, bars, and restaurants.

5. Northern Liberties: A former industrial area turned trendy neighborhood, Northern Liberties is known for its lively bar and restaurant scene, as well as its street art and unique shops.

6. Ches

# Sequential Chain

## Simple Sequential Chain

### Define the prompt templates for stay and itinerary suggestions

In [8]:
# Define the prompt templates
stay_prompt_template_name = PromptTemplate(
    input_variables=['city'],
    template="Can you suggest a nice place to stay in {city} for the weekend?"
)

itinerary_prompt_template_name = PromptTemplate(
    input_variables=['city'],
    template="Can you recommend a good travel itinerary for {city}?"
)

### Define the LLM Chains for stay and itinerary

In [9]:
# Define the LLM Chains
stay_chain = LLMChain(llm=llm, prompt=stay_prompt_template_name)
itinerary_chain = LLMChain(llm=llm, prompt=itinerary_prompt_template_name)

### Combine the chains into a Simple Sequential Chain

In [10]:
# Combine the chains into a SimpleSequentialChain
chain = SimpleSequentialChain(chains=[stay_chain, itinerary_chain])

### Prompt the user for input and run the chain

In [11]:
# Prompt the user for input
user_input = input("Enter a city: ")

# Run the chain with the user's input for the 'city' variable
response = chain.run(user_input)

# Printing the generated response to the console
print(response)

Enter a city:  LA




Day 1:
- Check into your hotel of choice and get settled in
- Head to Rodeo Drive in Beverly Hills for some luxury shopping and people watching
- Visit the iconic Hollywood Walk of Fame and take some photos with the stars
- Have dinner at one of LA's famous restaurants, such as Spago or Nobu

Day 2:
- Start your day with a hike to the Hollywood Sign for some amazing views of the city
- Visit the Getty Center for some art and culture
- Spend the afternoon at Santa Monica Pier, with its amusement park rides and beachfront views
- Have dinner at a trendy restaurant in Venice Beach, such as Gjelina or The Tasting Kitchen

Day 3:
- Take a tour of the Warner Bros. Studios or Universal Studios Hollywood for some behind-the-scenes Hollywood magic
- Explore the trendy neighborhoods of Silver Lake and Echo Park, known for their hip restaurants, bars, and street art
- Catch a show at the iconic Hollywood Bowl or a concert at the Staples Center
- End your trip with a night out in Downtown LA, ch

## Complex Sequential Chain

### Define the prompt templates for best times and attractions

In [12]:
# Define the prompt templates
times_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What are the best times of year to visit {city}?"
)

attractions_prompt_template = PromptTemplate(
    input_variables=['city', 'best_time'],
    template="Given that the best time to visit {city} is {best_time}, can you suggest the top attractions to see?"
)

### Define the LLM Chains for best times and attractions

In [13]:
# Define the LLM Chains
times_chain = LLMChain(llm=llm, prompt=times_prompt_template, output_key="best_time")
attractions_chain = LLMChain(llm=llm, prompt=attractions_prompt_template, output_key="top_attractions")

### Combine the chains into a SequentialChain

In [14]:
# Combine the chains into a SequentialChain
chain = SequentialChain(
    chains=[times_chain, attractions_chain],
    input_variables=['city'],
    output_variables=["best_time", "top_attractions"]
)

### Prompt the user for input and run the chain

In [15]:
# Prompt the user for input
user_input = input("Enter a city: ")

# Run the chain with the user's input for the 'city' variable
response = chain({"city": user_input})

# Printing the generated response to the console
print("Best time to visit:", response["best_time"])
print("Top attractions:", response["top_attractions"])

Enter a city:  Tokyo


C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Best time to visit: 

The best times of year to visit Tokyo are spring (March-May) and autumn (September-November). These seasons offer comfortable temperatures, beautiful cherry blossom and fall foliage viewing, and various cultural festivals and events. However, Tokyo can be enjoyed year-round and each season has its own unique charm. Summer (June-August) can be hot and humid, but it is also a great time to experience traditional summer festivals and enjoy outdoor activities. Winter (December-February) can be cold, but the city is adorned with beautiful holiday decorations and there are many indoor activities to enjoy.
Top attractions: 

1. Tokyo Tower - iconic landmark and observation deck with views of the city
2. Shibuya Crossing - famous intersection known for its crowds and neon lights
3. Meiji Shrine - peaceful Shinto shrine in the heart of the city
4. Sensoji Temple - oldest and most famous Buddhist temple in Tokyo
5. Tsukiji Fish Market - bustling market known for its fresh s

# Using an Agent

### Define the prompt templates

In [16]:
flight_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average cost of a flight to {city} from New York?"
)

accommodation_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average daily cost of accommodation in {city}?"
)

food_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average daily cost of food in {city}?"
)

activities_cost_prompt_template = PromptTemplate(
    input_variables=['city'],
    template="What is the average daily cost of activities in {city}?"
)

### Define the LLM Chains

In [17]:
flight_cost_chain = LLMChain(llm=llm, prompt=flight_cost_prompt_template)
accommodation_cost_chain = LLMChain(llm=llm, prompt=accommodation_cost_prompt_template)
food_cost_chain = LLMChain(llm=llm, prompt=food_cost_prompt_template)
activities_cost_chain = LLMChain(llm=llm, prompt=activities_cost_prompt_template)

### Load tools relevant for vacation planning

In [18]:
tools = load_tools(["wikipedia"], llm=llm)

### Initialize the agent with the necessary tools

In [19]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

C:\Users\ranal\anaconda3\envs\LLM\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


### Define a function to extract cost from response using regex

In [20]:
def extract_cost(response_text):
    costs = re.findall(r'\$\d+', response_text)
    costs = [int(cost.strip('$')) for cost in costs]
    if costs:
        return sum(costs) / len(costs)
    else:
        return 0  # Default to 0 if no costs are found

### Define a function to estimate costs

In [21]:
def estimate_costs(city):
    # Estimate costs using the Langchain agent
    flight_cost_response = flight_cost_chain.run({"city": city})
    accommodation_cost_response = accommodation_cost_chain.run({"city": city})
    food_cost_response = food_cost_chain.run({"city": city})
    activities_cost_response = activities_cost_chain.run({"city": city})

    flight_cost = extract_cost(flight_cost_response)
    accommodation_cost = extract_cost(accommodation_cost_response)
    food_cost = extract_cost(food_cost_response)
    activities_cost = extract_cost(activities_cost_response)

    # Calculate the total estimated cost for a week-long stay and a 3-day weekend
    total_week_cost = (
        flight_cost +
        (accommodation_cost * 7) +
        (food_cost * 7) +
        (activities_cost * 7)
    )

    total_weekend_cost = (
        flight_cost +
        (accommodation_cost * 3) +
        (food_cost * 3) +
        (activities_cost * 3)
    )

    # Display the results
    print(f"Estimated total cost for a week-long stay in {city}: ${total_week_cost:.2f}")
    print(f"Estimated total cost for a 3-day weekend stay in {city}: ${total_weekend_cost:.2f}")

### Example query for vacation planning

In [22]:
city = "Paris"
agent.run(f"I want to plan a vacation to {city}. What are the best times to visit and what is the estimated cost for a week-long stay?")

'The best times to visit Paris are in the spring and fall, and the estimated cost for a week-long stay can range from $1,500 to $3,000, depending on accommodations and activities.'

### Estimate costs

In [23]:
estimate_costs(city)

Estimated total cost for a week-long stay in Paris: $1925.00
Estimated total cost for a 3-day weekend stay in Paris: $1225.00
